In [ ]:

# Lookalike Customer Analysis
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

def prepare_customer_features(transactions, customers):
    # Aggregate customer behavior
    customer_features = transactions.groupby('CustomerID').agg({
        'TotalValue': ['sum', 'mean', 'std'],
        'Quantity': ['sum', 'mean', 'std'],
        'TransactionID': 'count'
    }).fillna(0)
    
    # Flatten column names
    customer_features.columns = ['_'.join(col).strip() for col in customer_features.columns.values]
    
    return customer_features

def find_similar_customers(customer_features, n_neighbors=5):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(customer_features)
    
    similarity_matrix = cosine_similarity(scaled_features)
    
    similar_customers = []
    for i in range(len(customer_features)):
        similar_indices = similarity_matrix[i].argsort()[-n_neighbors-1:-1][::-1]
        similarities = similarity_matrix[i][similar_indices]
        similar_customers.append(list(zip(similar_indices, similarities)))
    
    return similar_customers

# Main execution
customers, transactions = load_data()
customer_features = prepare_customer_features(transactions, customers)
similar_customers = find_similar_customers(customer_features)
